### __Make pseudo 27 point annotated imageset__

**Import modules**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import cv2
import random
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

import timm
from facenet_pytorch import MTCNN

from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = "cpu"

**Set data path**

In [ ]:
path = "/data/komedi/backup/opendataset/helen"
file_list = os.listdir(path)

name_list = []
for idx in (range(len(file_list))):
    name_list.append(os.path.join(path,file_list[idx]))
    
print(f"{len(name_list)} of images are detected \nDone.")

**Define model(facial landmark detection & face detection)**

In [ ]:
# Facial landmark detection model
def timm_Net_54(model_name, pretrained=None, num_classes=54):
    model = timm.create_model(
        model_name=model_name,
        pretrained=False,
        num_classes=num_classes
        )

    if not pretrained==None:
        model.eval()
        model.load_state_dict(torch.load(pretrained, map_location = 'cpu'), strict=False)
    
    return model

# Face detection model
def mtcnn(image, margin=200): #PIL
    numpy_image =  np.array(image)
    mtcnn = MTCNN(
    image_size=512, margin=margin, min_face_size=100,
    thresholds=[0.1,0.1,0.1], factor=0.5, post_process=True,
    device='cpu', select_largest=True
    )
    
    bbox, _ = mtcnn.detect(numpy_image)
    crop_area = (bbox[0][0]-margin//2, # get bbox area with margin
                 bbox[0][1]-margin//2,
                 bbox[0][2]+margin//2,
                 bbox[0][3]+margin//2)
    pil_image = Image.fromarray(numpy_image)
    crop_img = pil_image.crop(crop_area)
    
    return crop_img, bbox, np.array(crop_img).shape[0], np.array(crop_img).shape[1]

print("model is defined \nDone.")

In [ ]:
model_name = "swin_base_patch4_window7_224"
pretrained = "/data/komedi/pretrained_model/2022-07-08/frontal_27pt_pre0707_swin_base_patch4_window7_224_100.pt"
model = timm_Net_54(model_name, pretrained)

print("\n Done.")

**Run**

In [ ]:
def run(i):
    image = Image.open(name_list[i])

    crop_img, bbox, h, w = mtcnn(image)
    resize_img = TF.resize(crop_img, (224,224))
    image_tensor = TF.to_tensor(resize_img)
    image_tensor = image_tensor.unsqueeze(0)

    predict = model(image_tensor)

    landmarks = ((predict.view(-1,2)+0.5)).detach().numpy().tolist()
    landmarks = np.array([(int(x*w), int(y*h)) for (x, y) in landmarks if 0 <= x and 0 <= y])
    return landmarks, bbox, crop_img

print("\n Done.")

**Make pseudo-csv file**

In [ ]:
case = "helen"
csv_lists = []
for _, i in enumerate(tqdm(range(len(name_list)))):
    # control part
    # if i > 1:
    #     break
    
    try:
        csv_list = []
        
        if case == "kface":
            csv_list.append(os.path.splitext(file_list[i])[0])
            csv_list.append(file_list[i].split('_')[1])                
            csv_list.append(file_list[i].split('_')[2])
            csv_list.append(name_list[i])
        else:
            csv_list.append(os.path.splitext(file_list[i])[0])
            csv_list.append(f"/data/komedi/confirm_result/{case}/images_crop/{file_list[i]}")                
            csv_list.append('_')            
            csv_list.append(name_list[i])
            
        landmarks, bbox, crop_img = run(i)
        for bb in bbox[0]:
            csv_list.append(int(bb))
        for x,y in landmarks:
            csv_list.append((x,y))
        crop_img.save(f"/data/komedi/confirm_result/{case}/images_crop/{file_list[i]}")
        
        csv_lists.append(csv_list)
        df = pd.DataFrame(csv_lists)
        df.to_csv(f"/data/komedi/confirm_result/{case}/{case}_27pt_pseudo.csv", header=None, index=None)
                
    except TypeError:
        pass

print("\n Done.")

**Make pseudo-jpg file**

In [ ]:
pseudo_csv_path = f"/data/komedi/confirm_result/{case}/{case}_27pt_pseudo.csv"
pseudo_csv = pd.read_csv(pseudo_csv_path,header=None).values.tolist()

for _, pseudo in enumerate(tqdm(pseudo_csv)):
    image = cv2.cvtColor(cv2.imread(pseudo[1]),cv2.COLOR_BGR2RGB)
    landmarks = pseudo[8:]
    
    plt.figure(figsize=(10,10))
    for landmark in landmarks:
        x,y = eval(landmark)
        plt.scatter(x,y, s=10)
    plt.imshow(image)
    plt.axis("off")
    plt.savefig(f"/data/komedi/confirm_result/{case}/images/{pseudo[0]}.jpg", bbox_inches='tight',pad_inches = 0)

print("\n Done.")